---
title: "Loading Pre-trained Models"
subtitle: "Working with HuggingFace model hub"
jupyter: geoai
format: html
---


## Introduction to Pre-trained Models

Pre-trained models are essential for geospatial AI, providing foundation models trained on large satellite imagery datasets.


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoConfig, AutoImageProcessor
from huggingface_hub import hf_hub_download
import timm
import numpy as np
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")
print(f"Transformers available: {'✓' if 'transformers' in globals() else '✗'}")
print(f"Timm available: {'✓' if 'timm' in globals() else '✗'}")

## HuggingFace Model Hub

### Loading vision transformers

In [ ]:
# Example model configurations (adjust based on actual available models)
GEOSPATIAL_MODELS = {
    'prithvi': {
        'model_id': 'ibm-nasa-geospatial/Prithvi-100M',
        'description': 'NASA-IBM Prithvi foundation model for Earth observations'
    },
    'satlas': {
        'model_id': 'allenai/satlas-pretrain',  
        'description': 'Satlas foundation model for satellite imagery'
    },
    'clay': {
        'model_id': 'made-with-clay/Clay',
        'description': 'Clay foundation model for Earth observation'
    }
}

def load_geospatial_model(model_name='prithvi', device='cpu'):
    """Load a geospatial foundation model"""
    
    if model_name not in GEOSPATIAL_MODELS:
        print(f"Model {model_name} not found. Available: {list(GEOSPATIAL_MODELS.keys())}")
        return None
    
    model_info = GEOSPATIAL_MODELS[model_name]
    model_id = model_info['model_id']
    
    try:
        # Load configuration
        config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
        
        # Load model
        model = AutoModel.from_pretrained(
            model_id,
            config=config,
            trust_remote_code=True,
            torch_dtype=torch.float16 if device != 'cpu' else torch.float32
        )
        
        # Load image processor if available
        try:
            processor = AutoImageProcessor.from_pretrained(model_id, trust_remote_code=True)
        except:
            processor = None
            print(f"No image processor found for {model_name}")
        
        model.to(device)
        model.eval()
        
        print(f"Loaded {model_info['description']}")
        return model, processor, config
        
    except Exception as e:
        print(f"Error loading {model_name}: {e}")
        print("Creating mock model for demonstration...")
        
        # Create a mock transformer model for demonstration
        class MockGeospatialModel(nn.Module):
            def __init__(self, num_bands=6, embed_dim=768, num_patches=196):
                super().__init__()
                self.patch_embed = nn.Conv2d(num_bands, embed_dim, 16, 16)
                self.transformer = nn.TransformerEncoder(
                    nn.TransformerEncoderLayer(embed_dim, 12), 
                    num_layers=12
                )
                self.norm = nn.LayerNorm(embed_dim)
                
            def forward(self, x):
                # Patch embedding
                x = self.patch_embed(x)  # [B, embed_dim, H/16, W/16]
                B, C, H, W = x.shape
                x = x.flatten(2).transpose(1, 2)  # [B, num_patches, embed_dim]
                
                # Add positional embeddings (simplified)
                x = self.transformer(x)
                x = self.norm(x)
                
                return {'last_hidden_state': x}
        
        model = MockGeospatialModel()
        model.to(device)
        model.eval()
        
        return model, None, {'description': f'Mock {model_name} model'}

# Load a geospatial model
model, processor, config = load_geospatial_model('prithvi', device='cpu')

if model is not None:
    print(f"Model loaded successfully")
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

### Model configuration and metadata

In [ ]:
def inspect_model_config(model, config):
    """Inspect model configuration and capabilities"""
    
    print("=== Model Configuration ===")
    
    if hasattr(config, '__dict__'):
        for key, value in config.__dict__.items():
            if not key.startswith('_'):
                print(f"{key}: {value}")
    
    print("\n=== Model Architecture ===")
    
    # Count parameters by layer type
    layer_params = {}
    for name, module in model.named_modules():
        layer_type = type(module).__name__
        if layer_type not in layer_params:
            layer_params[layer_type] = 0
        layer_params[layer_type] += sum(p.numel() for p in module.parameters())
    
    for layer_type, params in sorted(layer_params.items(), key=lambda x: x[1], reverse=True):
        if params > 0:
            print(f"{layer_type}: {params:,} parameters")

if model is not None and config is not None:
    inspect_model_config(model, config)

### Custom model loading utilities

In [ ]:
class GeospatialModelLoader:
    """Utility class for loading geospatial models"""
    
    def __init__(self, cache_dir='./model_cache'):
        self.cache_dir = cache_dir
        self.loaded_models = {}
    
    def download_model_file(self, model_id, filename, subfolder=None):
        """Download specific model file"""
        try:
            file_path = hf_hub_download(
                repo_id=model_id,
                filename=filename,
                subfolder=subfolder,
                cache_dir=self.cache_dir
            )
            print(f"Downloaded {filename} to {file_path}")
            return file_path
        except Exception as e:
            print(f"Error downloading {filename}: {e}")
            return None
    
    def load_with_custom_weights(self, model_id, local_weights_path=None):
        """Load model with custom weights"""
        
        # Create base model architecture
        try:
            config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)
            model = AutoModel.from_pretrained(
                model_id, 
                config=config, 
                trust_remote_code=True
            )
            
            # Load custom weights if provided
            if local_weights_path and os.path.exists(local_weights_path):
                print(f"Loading custom weights from {local_weights_path}")
                state_dict = torch.load(local_weights_path, map_location='cpu')
                model.load_state_dict(state_dict, strict=False)
            
            return model, config
            
        except Exception as e:
            print(f"Error loading model: {e}")
            return None, None
    
    def cache_model(self, model_name, model, config):
        """Cache loaded model in memory"""
        self.loaded_models[model_name] = {
            'model': model,
            'config': config,
            'timestamp': torch.tensor(0)  # Placeholder timestamp
        }
    
    def get_cached_model(self, model_name):
        """Retrieve cached model"""
        return self.loaded_models.get(model_name, None)

# Initialize model loader
loader = GeospatialModelLoader()

# Example usage
# custom_model, custom_config = loader.load_with_custom_weights(
#     'ibm-nasa-geospatial/Prithvi-100M',
#     local_weights_path='path/to/custom_weights.pth'
# )

print("Model loader utilities ready")

## TIMM (Torch Image Models) Integration

### Loading vision models from TIMM

In [ ]:
def load_timm_model(model_name, pretrained=True, num_classes=1000):
    """Load vision model from TIMM"""
    
    try:
        # List some common geospatial-friendly architectures
        if model_name == 'demo':
            model_name = 'resnet50'  # Use ResNet50 for demo
        
        model = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=num_classes,
            global_pool='avg'  # Global average pooling
        )
        
        # Get model information
        model_info = {
            'name': model_name,
            'num_params': sum(p.numel() for p in model.parameters()),
            'input_size': model.default_cfg.get('input_size', (3, 224, 224)),
            'mean': model.default_cfg.get('mean', (0.485, 0.456, 0.406)),
            'std': model.default_cfg.get('std', (0.229, 0.224, 0.225))
        }
        
        return model, model_info
        
    except Exception as e:
        print(f"Error loading TIMM model {model_name}: {e}")
        return None, None

# Load a TIMM model
model_timm, model_info = load_timm_model('demo', pretrained=True, num_classes=10)

if model_timm is not None:
    print(f"TIMM Model: {model_info['name']}")
    print(f"Parameters: {model_info['num_params']:,}")
    print(f"Input size: {model_info['input_size']}")
    print(f"Normalization - Mean: {model_info['mean']}")
    print(f"Normalization - Std: {model_info['std']}")

### Adapting models for multi-spectral data

In [ ]:
class MultispectralAdapter:
    """Adapter for converting RGB models to multi-spectral"""
    
    @staticmethod
    def adapt_input_layer(model, num_input_channels=6, method='repeat'):
        """Adapt first layer for multi-spectral input"""
        
        # Find first convolutional layer
        first_conv = None
        first_conv_name = None
        
        for name, module in model.named_modules():
            if isinstance(module, nn.Conv2d):
                first_conv = module
                first_conv_name = name
                break
        
        if first_conv is None:
            print("No convolutional layer found")
            return model
        
        old_weight = first_conv.weight.data
        old_in_channels = old_weight.shape[1]
        
        print(f"Adapting {first_conv_name}: {old_in_channels} -> {num_input_channels} channels")
        
        if method == 'repeat':
            # Repeat RGB weights for additional channels
            if num_input_channels > old_in_channels:
                # Calculate how to distribute new channels
                repeats = num_input_channels // old_in_channels
                remainder = num_input_channels % old_in_channels
                
                new_weight = old_weight.repeat(1, repeats, 1, 1)
                if remainder > 0:
                    extra_weight = old_weight[:, :remainder, :, :]
                    new_weight = torch.cat([new_weight, extra_weight], dim=1)
                
                # Scale weights to maintain similar activation magnitudes
                new_weight = new_weight / (num_input_channels / old_in_channels)
                
            else:
                # Use subset of weights
                new_weight = old_weight[:, :num_input_channels, :, :]
        
        elif method == 'zeros':
            # Initialize new channels with zeros
            new_weight = torch.zeros(
                old_weight.shape[0], num_input_channels,
                old_weight.shape[2], old_weight.shape[3]
            )
            # Copy RGB weights to first channels
            copy_channels = min(old_in_channels, num_input_channels)
            new_weight[:, :copy_channels, :, :] = old_weight[:, :copy_channels, :, :]
        
        elif method == 'random':
            # Random initialization for new channels
            new_weight = torch.randn(
                old_weight.shape[0], num_input_channels,
                old_weight.shape[2], old_weight.shape[3]
            ) * 0.01
            # Copy RGB weights
            copy_channels = min(old_in_channels, num_input_channels)
            new_weight[:, :copy_channels, :, :] = old_weight[:, :copy_channels, :, :]
        
        # Create new layer
        new_conv = nn.Conv2d(
            num_input_channels,
            first_conv.out_channels,
            first_conv.kernel_size,
            first_conv.stride,
            first_conv.padding,
            first_conv.dilation,
            first_conv.groups,
            first_conv.bias is not None
        )
        
        # Set weights and bias
        new_conv.weight.data = new_weight
        if first_conv.bias is not None:
            new_conv.bias.data = first_conv.bias.data
        
        # Replace layer in model
        parent_module = model
        attr_chain = first_conv_name.split('.')
        
        for attr_name in attr_chain[:-1]:
            parent_module = getattr(parent_module, attr_name)
        
        setattr(parent_module, attr_chain[-1], new_conv)
        
        return model
    
    @staticmethod
    def freeze_backbone(model, freeze_layers=['conv1', 'layer1', 'layer2']):
        """Freeze specified layers of the model"""
        
        frozen_params = 0
        total_params = 0
        
        for name, param in model.named_parameters():
            total_params += param.numel()
            
            should_freeze = any(layer in name for layer in freeze_layers)
            if should_freeze:
                param.requires_grad = False
                frozen_params += param.numel()
        
        print(f"Frozen {frozen_params:,} / {total_params:,} parameters")
        print(f"Frozen layers: {freeze_layers}")
        
        return model

# Example: Adapt ResNet for 6-channel input (RGB + NIR + SWIR1 + SWIR2)
if model_timm is not None:
    print("Original model input shape:", model_timm.conv1.weight.shape)
    
    # Adapt for multispectral input
    adapter = MultispectralAdapter()
    model_adapted = adapter.adapt_input_layer(model_timm, num_input_channels=6, method='repeat')
    
    print("Adapted model input shape:", model_adapted.conv1.weight.shape)
    
    # Freeze early layers
    model_adapted = adapter.freeze_backbone(model_adapted, ['conv1', 'layer1'])

## Model Input Preprocessing

### Creating preprocessing pipelines

In [ ]:
class SatellitePreprocessor:
    """Preprocessing pipeline for satellite imagery"""
    
    def __init__(self, 
                 input_size=(224, 224),
                 mean=[0.485, 0.456, 0.406, 0.5, 0.5, 0.5],  # RGB + additional bands
                 std=[0.229, 0.224, 0.225, 0.2, 0.2, 0.2],
                 clip_values=None):
        
        self.input_size = input_size
        self.mean = torch.tensor(mean)
        self.std = torch.tensor(std)
        self.clip_values = clip_values
    
    def __call__(self, image):
        """Apply preprocessing to image tensor"""
        
        # Ensure tensor format [C, H, W]
        if image.ndim == 4:
            image = image.squeeze(0)
        
        # Resize if needed
        if image.shape[-2:] != self.input_size:
            image = torch.nn.functional.interpolate(
                image.unsqueeze(0), 
                size=self.input_size, 
                mode='bilinear', 
                align_corners=False
            ).squeeze(0)
        
        # Clip values if specified
        if self.clip_values:
            image = torch.clamp(image, self.clip_values[0], self.clip_values[1])
        
        # Normalize
        if len(self.mean) >= image.shape[0]:
            mean = self.mean[:image.shape[0]]
            std = self.std[:image.shape[0]]
            
            # Reshape for broadcasting
            mean = mean.view(-1, 1, 1)
            std = std.view(-1, 1, 1)
            
            image = (image - mean) / std
        
        return image
    
    def batch_preprocess(self, batch):
        """Preprocess a batch of images"""
        if isinstance(batch, list):
            return torch.stack([self(img) for img in batch])
        elif batch.ndim == 4:
            return torch.stack([self(batch[i]) for i in range(batch.shape[0])])
        else:
            return self(batch)

# Create preprocessor
preprocessor = SatellitePreprocessor(
    input_size=(224, 224),
    mean=[0.485, 0.456, 0.406, 0.4, 0.3, 0.2],  # Adjusted for 6 bands
    std=[0.229, 0.224, 0.225, 0.15, 0.12, 0.10],
    clip_values=[0, 1]
)

# Test preprocessing
sample_image = torch.rand(6, 256, 256)  # 6-band image
processed_image = preprocessor(sample_image)

print(f"Original shape: {sample_image.shape}")
print(f"Processed shape: {processed_image.shape}")
print(f"Processed range: [{processed_image.min():.3f}, {processed_image.max():.3f}]")

### Band selection and configuration

In [ ]:
class BandSelector:
    """Utility for selecting and configuring satellite bands"""
    
    BAND_CONFIGS = {
        'landsat8': {
            'bands': ['coastal', 'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pan', 'cirrus', 'thermal1', 'thermal2'],
            'rgb': [3, 2, 1],  # Red, Green, Blue (1-indexed)
            'false_color': [4, 3, 2],  # NIR, Red, Green
            'swir': [6, 5, 4]  # SWIR2, SWIR1, NIR
        },
        'sentinel2': {
            'bands': ['coastal', 'blue', 'green', 'red', 'red_edge1', 'red_edge2', 'red_edge3', 'nir', 'red_edge4', 'water_vapor', 'swir1', 'swir2'],
            'rgb': [3, 2, 1],
            'false_color': [7, 3, 2],
            'agriculture': [7, 5, 2]
        }
    }
    
    def __init__(self, satellite='landsat8'):
        self.satellite = satellite
        self.config = self.BAND_CONFIGS.get(satellite, {})
    
    def select_bands(self, image, band_combination='rgb'):
        """Select specific band combination"""
        
        if band_combination not in self.config:
            print(f"Unknown combination: {band_combination}")
            print(f"Available: {list(self.config.keys())}")
            return image
        
        band_indices = self.config[band_combination]
        # Convert to 0-indexed
        selected_indices = [i - 1 for i in band_indices]
        
        # Select bands
        if image.ndim == 3:  # [C, H, W]
            selected_image = image[selected_indices]
        elif image.ndim == 4:  # [B, C, H, W]
            selected_image = image[:, selected_indices]
        else:
            print(f"Unsupported image shape: {image.shape}")
            return image
        
        return selected_image
    
    def get_band_names(self, band_combination='rgb'):
        """Get names of selected bands"""
        if band_combination in self.config:
            indices = self.config[band_combination]
            return [self.config['bands'][i-1] for i in indices]
        return []

# Example usage
selector = BandSelector('landsat8')

# Simulate 11-band Landsat image
landsat_image = torch.rand(11, 256, 256)

# Select RGB combination
rgb_image = selector.select_bands(landsat_image, 'rgb')
false_color_image = selector.select_bands(landsat_image, 'false_color')

print(f"Original image: {landsat_image.shape}")
print(f"RGB selection: {rgb_image.shape}")
print(f"RGB bands: {selector.get_band_names('rgb')}")
print(f"False color bands: {selector.get_band_names('false_color')}")

## Model Inference Setup

### Inference wrapper class

In [ ]:
class GeospatialInference:
    """Wrapper for geospatial model inference"""
    
    def __init__(self, model, preprocessor=None, device='cpu'):
        self.model = model
        self.preprocessor = preprocessor
        self.device = device
        self.model.to(device)
        self.model.eval()
    
    @torch.no_grad()
    def predict(self, images, return_features=False):
        """Run inference on images"""
        
        # Handle single image or batch
        if isinstance(images, torch.Tensor):
            if images.ndim == 3:  # Single image [C, H, W]
                images = images.unsqueeze(0)  # Add batch dim
            batch = images
        elif isinstance(images, list):
            batch = torch.stack(images)
        else:
            raise ValueError(f"Unsupported input type: {type(images)}")
        
        # Preprocess if preprocessor available
        if self.preprocessor:
            batch = self.preprocessor.batch_preprocess(batch)
        
        # Move to device
        batch = batch.to(self.device)
        
        # Forward pass
        if hasattr(self.model, 'forward_features'):
            # TIMM-style model
            features = self.model.forward_features(batch)
            logits = self.model.forward_head(features)
            
            if return_features:
                return logits, features
            return logits
            
        else:
            # Standard forward pass
            outputs = self.model(batch)
            
            # Handle different output formats
            if isinstance(outputs, dict):
                if 'last_hidden_state' in outputs:
                    features = outputs['last_hidden_state']
                    # Global average pooling for classification
                    if len(features.shape) == 3:  # [B, seq_len, dim]
                        pooled = features.mean(dim=1)
                    else:
                        pooled = features
                    
                    if return_features:
                        return pooled, features
                    return pooled
                else:
                    return outputs
            else:
                return outputs
    
    def extract_features(self, images, layer_name=None):
        """Extract features from specific layer"""
        
        features = {}
        
        def hook(name):
            def fn(module, input, output):
                features[name] = output.detach()
            return fn
        
        # Register hooks
        hooks = []
        for name, module in self.model.named_modules():
            if layer_name is None or layer_name in name:
                hooks.append(module.register_forward_hook(hook(name)))
        
        # Forward pass
        _ = self.predict(images)
        
        # Remove hooks
        for hook in hooks:
            hook.remove()
        
        return features
    
    def get_attention_maps(self, images, layer_idx=-1):
        """Extract attention maps (for transformer models)"""
        
        # This is a simplified example - actual implementation
        # depends on specific model architecture
        
        attention_maps = {}
        
        def attention_hook(name):
            def fn(module, input, output):
                # Assuming output contains attention weights
                if hasattr(output, 'attentions'):
                    attention_maps[name] = output.attentions
                elif len(output) > 1 and hasattr(output[1], 'shape'):
                    attention_maps[name] = output[1]  # Attention weights
            return fn
        
        # Look for attention modules
        hooks = []
        for name, module in self.model.named_modules():
            if 'attention' in name.lower() or 'attn' in name.lower():
                hooks.append(module.register_forward_hook(attention_hook(name)))
        
        # Forward pass
        _ = self.predict(images)
        
        # Remove hooks
        for hook in hooks:
            hook.remove()
        
        return attention_maps if attention_maps else None

# Create inference wrapper
if model_adapted is not None and preprocessor is not None:
    inference = GeospatialInference(model_adapted, preprocessor, device='cpu')
    
    # Test inference
    test_image = torch.rand(6, 256, 256)  # 6-band image
    predictions = inference.predict(test_image)
    
    print(f"Input shape: {test_image.shape}")
    print(f"Prediction shape: {predictions.shape}")
    print("Inference wrapper ready")
else:
    print("Model or preprocessor not available for inference demo")

### Batch inference utilities

In [ ]:
class BatchInference:
    """Utilities for efficient batch inference"""
    
    def __init__(self, model, batch_size=32, device='cpu'):
        self.model = model
        self.batch_size = batch_size
        self.device = device
        self.model.to(device)
        self.model.eval()
    
    @torch.no_grad()
    def predict_dataset(self, dataloader, progress=True):
        """Run inference on entire dataset"""
        
        all_predictions = []
        all_labels = []
        
        total_batches = len(dataloader) if hasattr(dataloader, '__len__') else None
        
        for batch_idx, batch in enumerate(dataloader):
            if progress and total_batches:
                print(f"Processing batch {batch_idx + 1}/{total_batches}", end='\r')
            
            # Handle different batch formats
            if isinstance(batch, dict):
                images = batch['image'].to(self.device)
                if 'label' in batch:
                    labels = batch['label']
                    all_labels.extend(labels.cpu().numpy())
            else:
                images = batch[0].to(self.device) if isinstance(batch, (list, tuple)) else batch.to(self.device)
            
            # Forward pass
            outputs = self.model(images)
            
            # Handle different output formats
            if isinstance(outputs, dict):
                predictions = outputs.get('logits', outputs.get('last_hidden_state', outputs))
            else:
                predictions = outputs
            
            all_predictions.append(predictions.cpu())
        
        # Concatenate all predictions
        final_predictions = torch.cat(all_predictions, dim=0)
        
        result = {'predictions': final_predictions}
        if all_labels:
            result['labels'] = np.array(all_labels)
        
        if progress:
            print(f"\nCompleted inference on {len(final_predictions)} samples")
        
        return result
    
    def predict_large_image(self, large_image, window_size=224, stride=112, aggregation='mean'):
        """Run inference on large image using sliding windows"""
        
        if large_image.ndim == 3:  # [C, H, W]
            c, h, w = large_image.shape
        else:
            raise ValueError("Expected 3D image [C, H, W]")
        
        # Calculate number of windows
        h_windows = (h - window_size) // stride + 1
        w_windows = (w - window_size) // stride + 1
        
        predictions = []
        window_positions = []
        
        for i in range(h_windows):
            for j in range(w_windows):
                # Extract window
                start_h = i * stride
                end_h = start_h + window_size
                start_w = j * stride  
                end_w = start_w + window_size
                
                window = large_image[:, start_h:end_h, start_w:end_w]
                
                # Predict on window
                window_batch = window.unsqueeze(0).to(self.device)
                output = self.model(window_batch)
                
                predictions.append(output.cpu())
                window_positions.append((start_h, end_h, start_w, end_w))
        
        return {
            'predictions': predictions,
            'positions': window_positions,
            'grid_shape': (h_windows, w_windows)
        }

# Example usage with mock data
if model_adapted is not None:
    batch_inference = BatchInference(model_adapted, batch_size=16, device='cpu')
    
    # Simulate large image inference
    large_test_image = torch.rand(6, 512, 512)
    results = batch_inference.predict_large_image(
        large_test_image, 
        window_size=224, 
        stride=112
    )
    
    print(f"Large image shape: {large_test_image.shape}")
    print(f"Number of windows: {len(results['predictions'])}")
    print(f"Grid shape: {results['grid_shape']}")

## Summary

Key concepts for loading pre-trained models:
- **HuggingFace Hub**: Access to geospatial foundation models
- **TIMM Integration**: Computer vision models for satellite imagery
- **Multi-spectral adaptation**: Adapting RGB models for satellite bands
- **Preprocessing pipelines**: Normalization and band selection
- **Inference wrappers**: Efficient model prediction
- **Batch processing**: Large-scale inference utilities
- **Feature extraction**: Accessing intermediate model representations